# DistilBERT - Training on GoEmotions (Ran on Colab)

## Install requirements and import packages

In [1]:
!pip install pandas numpy scikit-learn torch transformers datasets


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import torch
import numpy as np

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoConfig,
    DistilBertForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    EarlyStoppingCallback
)
from sklearn.metrics import f1_score
from datasets import DatasetDict, Sequence, Value
import numpy as np
import torch
from sklearn.metrics import f1_score, accuracy_score
from torch.ao.nn.quantized.functional import threshold

/Users/sanshritbakshi/PycharmProjects/CS6120_final/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Loading and preparing Dataset

In [3]:
# Set the device based on environment.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# If locally
"""
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
"""
# load the GoEmotions “simplified” split
dataset = load_dataset("go_emotions", "simplified")
label_names = dataset["train"].features["labels"].feature.names
num_labels = len(label_names)
print(f"{num_labels} emotion labels:", label_names)

28 emotion labels: ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']


In [4]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_data(text):
    """
    Tokenize a batch of examples from GoEmotions (or any similar dataset).

    Args:
        examples (dict[str, list[str]]): a batch of examples, e.g.
            { "text": ["I love this!", "So sad today"], ... }

    Returns:
        dict[str, list[list[int]]]: a dict containing
            - input_ids: List of token IDs
            - attention_mask: List of attention masks
    """
    return tokenizer(text["text"], padding="max_length", truncation=True, max_length=128)

tokenize_dataset = dataset.map(tokenize_data, batched=True)
print(tokenize_dataset.column_names)

Map: 100%|██████████| 5427/5427 [00:00<00:00, 49152.74 examples/s]

{'train': ['text', 'labels', 'id', 'input_ids', 'attention_mask'], 'validation': ['text', 'labels', 'id', 'input_ids', 'attention_mask'], 'test': ['text', 'labels', 'id', 'input_ids', 'attention_mask']}


In [5]:
def process_labels(example):
    """
    Convert a multi‐label example’s integer label list into a multi‐hot float vector.

    This is meant to be used with a Dataset.map call on GoEmotions (or any
    multi‐label dataset), turning the “labels” field from a list of indices
    into a fixed‐length list of 0.0/1.0 floats for BCEWithLogitsLoss.

    Args:
        example (dict): A single data point dict with keys at least:
            - "labels": List[int], the indices of all positive emotion labels.

    Returns:
        dict: The same example dict, but with:
            - example["labels"] now a List[float] of length len(label_names),
              where positions in the original example["labels"] are set to 1.0
              and all others to 0.0.
    """
    vec = [0.0] * len(label_names)
    for idx in example["labels"]:
        vec[idx] = 1.0
    example["labels"] = vec
    return example

#Apply tokenization (already in `tokenize_dataset`) + label mapping
processed = tokenize_dataset.map(process_labels, batched=False)

# Ensure 'labels' is stored as float32
processed = processed.cast_column(
    "labels",
    Sequence(Value("float32"))
)

# Now tell Datasets to hand you torch.Tensors on the fly
columns = ["input_ids", "attention_mask", "labels"]
for split in ["train", "validation", "test"]:
    processed[split].set_format(type="torch", columns=columns)

# Assign back to your variables
train_dataset = processed["train"]
val_dataset   = processed["validation"]
test_dataset  = processed["test"]

# Sanity check
sample = train_dataset[0]
print("input_ids dtype:      ", sample["input_ids"].dtype)       # torch.int64
print("attention_mask dtype: ", sample["attention_mask"].dtype)  # torch.int64
print("labels dtype:         ", sample["labels"].dtype)          # torch.float32

Casting the dataset: 100%|██████████| 5427/5427 [00:00<00:00, 1031750.88 examples/s]

input_ids dtype:       torch.int64
attention_mask dtype:  torch.int64
labels dtype:          torch.float32


## Loading DistilBERT and Training

In [6]:
def compute_metrics(eval_pred):
    """
    Compute evaluation metrics for multi-label classification.

    Args:
        eval_pred (tuple):
            A tuple of (logits, labels)
            - logits: np.ndarray of shape (batch_size, num_labels)
              Raw outputs from the model’s classification head.
            - labels: np.ndarray of shape (batch_size, num_labels)
              Ground-truth multi-hot vectors (0/1).
        threshold (float, optional):
            Probability cutoff for deciding positive labels after sigmoid.
            Defaults to 0.3.

    Returns:
        dict:
            {
                "f1_micro": float,
                    The micro-averaged F1 score across all labels.
                "subset_accuracy": float,
                    The fraction of samples where the predicted multi-hot
                    vector exactly matches the ground truth.
            }
    """
    logits, labels = eval_pred

    # Convert logits to probabilities
    probs = torch.sigmoid(torch.tensor(logits))

    # Binarize predictions at 0.5
    preds = (probs > 0.3).int().numpy()
    labels = torch.tensor(labels).int().numpy()

    # Micro F1 score
    f1_micro = f1_score(labels, preds, average="micro")
    subset_acc = np.mean(np.all(preds == labels, axis=1))

    return {
        "f1_micro": f1_micro,
        "subset_accuracy": subset_acc
    }

In [ ]:
config  = AutoConfig.from_pretrained(
    "distilbert-base-uncased",
    num_labels=len(label_names),
    problem_type="multi_label_classification"
)
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    config=config
).to(device)

#Data collator
data_collator = DataCollatorWithPadding(tokenizer)

# TrainingArguments
training_args = TrainingArguments(
    output_dir="./distil_rebaseline",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1_micro",
    greater_is_better=True,

    num_train_epochs=9,
    per_device_train_batch_size=16,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",

    fp16=torch.cuda.is_available(),
    logging_strategy="epoch",
    report_to="none",
)

#Trainer with early stopping.
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)],
)

#Train
trainer.train()

# Evaluate
val_metrics = trainer.evaluate()
print("Validation micro‑F1:", val_metrics["eval_f1_micro"])

test_out = trainer.predict(test_dataset)
test_metrics = compute_metrics((test_out.predictions, test_out.label_ids))
print("Test micro‑F1:", test_metrics["f1_micro"])

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/xk/v43z00vn3mj3lb_2ryh_5wxm0000gn/T/ipykernel_20305/3060264966.py:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,F1 Micro,Subset Accuracy
1,0.141300,0.094930,0.564518,0.440472


/Users/sanshritbakshi/PycharmProjects/CS6120_final/.venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Validation micro‑F1: 0.5645175174313506


/Users/sanshritbakshi/PycharmProjects/CS6120_final/.venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Test micro‑F1: 0.5692214217515841


## Save the model and Tokenizer

In [ ]:
model.save_pretrained("./DistilBERT")
tokenizer.save_pretrained("./DistilBERT")

('./DBERT/tokenizer_config.json',
 './DBERT/special_tokens_map.json',
 './DBERT/vocab.txt',
 './DBERT/added_tokens.json',
 './DBERT/tokenizer.json')

In [ ]:
!zip -r DistilBERT.zip ./DistilBERT

  adding: DBERT/ (stored 0%)
  adding: DBERT/tokenizer_config.json (deflated 75%)
  adding: DBERT/model.safetensors (deflated 8%)
  adding: DBERT/special_tokens_map.json (deflated 42%)
  adding: DBERT/config.json (deflated 65%)
  adding: DBERT/vocab.txt (deflated 53%)
  adding: DBERT/tokenizer.json (deflated 71%)


## Upload model to HuggingFace
-  use the command - "huggingface-cli login"
- input the API token key (create a write token from huggingFace)
- Create a repo for model
- load the model from local file or use trained model

## Alternatively
- Unzip the files and upload to Repo created on HuggingFace.

In [ ]:
# Push the model
model.push_to_hub("Username/ModelRepo", commit_message="Initial model upload")

# Push the tokenizer
tokenizer.push_to_hub("Username/ModelRepo", commit_message="Initial tokenizer upload")

## Test functionality manually with text inputs

In [13]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
def predict_emotions(text, threshold=0.5):
    """
    Predict emotions for a given input text.

    Args:
        text (str): The input text to analyze.
        threshold (float): The probability threshold to decide if an emotion is present.

    Returns:
        predicted_emotions (list): List of emotion names predicted for the input text.
        probs (ndarray): Array of probability scores for each emotion.
    """
    # Ensure the model is in evaluation mode
    model.eval()

    # Tokenize the input text with same parameters used during training
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=128
    )

    # Move input tensors to the correct device (CPU/GPU/MPS)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Perform a forward pass without gradient calculation
    with torch.no_grad():
        outputs = model(**inputs)
        # Outputs logits from the model's classification head
        logits = outputs.logits

        # Apply sigmoid activation to convert logits to probabilities
        probs = torch.sigmoid(logits)[0].cpu().numpy()

    # Select labels where the probability exceeds the threshold
    predicted_emotions = [label_names[i] for i, prob in enumerate(probs) if prob > threshold]

    return predicted_emotions, probs

# Example usage:
text_input = "I am a CS student."
emotions, probabilities = predict_emotions(text_input, threshold=0.3)  # You may adjust threshold

print("Input text:", text_input)
print("Predicted Emotions:", emotions)
print("Raw Probabilities:", probabilities)

Input text: I am a CS student with no job
Predicted Emotions: ['neutral']
Raw Probabilities: [0.00915589 0.00610763 0.01685226 0.05316507 0.06666031 0.01489406
 0.01011656 0.00340904 0.00464192 0.02311579 0.09981031 0.00837776
 0.00322128 0.00437845 0.00561674 0.00204492 0.00107088 0.00438766
 0.00361852 0.00162727 0.00606414 0.00150995 0.02774598 0.00186503
 0.00222169 0.01016396 0.00551558 0.7070919 ]


# Results

In [15]:
import numpy as np
import torch
from sklearn.metrics import (
    f1_score,
    hamming_loss,
    classification_report,
    multilabel_confusion_matrix
)
import pandas as pd

#Get raw predictions and gold labels
test_out = trainer.predict(test_dataset)
logits   = test_out.predictions           # shape (N, num_labels)
y_true   = test_out.label_ids             # shape (N, num_labels), multi-hot

#Binarize using sigmoid + threshold
threshold = 0.3
probs = torch.sigmoid(torch.tensor(logits)).numpy()  # (N, num_labels)
y_pred = (probs > threshold).astype(int)             # (N, num_labels)

#Overall metrics
micro_f1      = f1_score(   y_true, y_pred, average="micro")
subset_acc    = np.mean((y_true == y_pred).all(axis=1))
hamming_acc   = 1 - hamming_loss(y_true, y_pred)

print(f"Micro-F1         : {micro_f1:.4f}")
print(f"Subset accuracy  : {subset_acc:.4f}")
print(f"Hamming accuracy : {hamming_acc:.4f}\n")

# Full classification report
print("Per-class classification report:")
print(classification_report(
    y_true,
    y_pred,
    target_names=label_names,
    zero_division=0
))

#Multi-label confusion matrices
mcm = multilabel_confusion_matrix(y_true, y_pred)
conf_df = pd.DataFrame(
    [cm.ravel() for cm in mcm],
    columns=["TN", "FP", "FN", "TP"],
    index=label_names
)
print("\nPer-class confusion stats (TN, FP, FN, TP):")
display(conf_df)

/Users/sanshritbakshi/PycharmProjects/CS6120_final/.venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Micro-F1         : 0.5692
Subset accuracy  : 0.4385
Hamming accuracy : 0.9673

Per-class classification report:
                precision    recall  f1-score   support

    admiration       0.64      0.72      0.68       504
     amusement       0.78      0.88      0.83       264
         anger       0.55      0.32      0.41       198
     annoyance       0.59      0.05      0.10       320
      approval       0.59      0.22      0.32       351
        caring       0.00      0.00      0.00       135
     confusion       0.71      0.07      0.12       153
     curiosity       0.49      0.70      0.58       284
        desire       0.70      0.08      0.15        83
disappointment       0.00      0.00      0.00       151
   disapproval       0.56      0.09      0.16       267
       disgust       0.00      0.00      0.00       123
 embarrassment       0.00      0.00      0.00        37
    excitement       0.00      0.00      0.00       103
          fear       0.00      0.00      0.00  

,TN,FP,FN,TP
admiration,4720,203,142,362
amusement,5098,65,32,232
anger,5177,52,134,64
annoyance,5095,12,303,17
approval,5021,55,273,78
caring,5292,0,135,0
confusion,5270,4,143,10
curiosity,4936,207,84,200
desire,5341,3,76,7
disappointment,5276,0,151,0
